In [ ]:
# default_exp likelihoodMethods

In [ ]:
import autograd
from autograd import grad,jacobian,hessian
from autograd.scipy import stats as agss
import autograd.numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import scipy.stats as ss
import os
from scipy.optimize import minimize
from glob import glob

In [ ]:
os.sched_setaffinity(0,set(range(10,20)))

In [ ]:
os.sched_getaffinity(0)

In [ ]:
def negativeLogLikelihood(x):
    def nLL(theta):
        ll = 0.0
        for i in range(len(x)):
            ll = ll + (1/len(x[i]))*(-len(x[i])/2 * np.log(2*np.pi*theta[i*2+1]**2) - (1/(2*theta[i*2+1]**2)) * np.sum((x[i] - theta[i*2])**2))
        return -1 * ll
    return nLL


In [ ]:
muStar = np.array([3.5,4,3,3,5,1,5])
sigmaStar = np.array([.5,1,1,2,2,2,2])
N = [400,200,200,100,100,100,100]
x = [np.random.normal(loc=mu,scale=sigma,size=(1,ni)) for mu,sigma,ni in zip(muStar,sigmaStar,N)]

In [ ]:
len(x)

# Autograd

In [ ]:
# theta = np.random.beta(a=1,b=1,size=(4,))
theta = np.array([0.0, 1.0,
                  0.0,1.0, 0.0,1.0,
                  0.0,1.0, 0.0,1.0, 0.0,1.0, 0.0,1.0])
gradTheta = grad(negativeLogLikelihood(x),)

maes = []
for i in range(1500):
    theta = theta - 0.001 * gradTheta(theta)
    aes = 0
    for i in range(int(len(theta)/2)):
        aes += np.abs(theta[2*i] - muStar[i])
    maes.append(aes/(len(theta)/2))

# jacobian_ = jacobian(negativeLogLikelihood(x))
# hessian_ = hessian(negativeLogLikelihood(x))
# for i in range(1000):
#     j = jacobian_(theta)
#     h = hessian_(theta)
#     theta = theta + 0.001 * np.linalg.inv(h) @ j
#     aes = np.abs(theta[0] - muStar[0]) + np.abs(theta[2] - muStar[1])
#     maes.append(aes)

In [ ]:
plt.plot(maes)

# In terms of Bag Estimates

In [ ]:
def logLikelihood(xi,mu,sigma,normalize):
    LL = (-len(xi)/2 * np.log(2*np.pi*(sigma + 1e-8)**2) - (1/(2*(sigma + 1e-8)**2)) * np.sum((xi - mu)**2))
    if normalize:
        LL = LL * (1/len(xi))
    return LL

def getChildren(idx,N):
    if idx > N - 1:
        return np.array([idx])
    left = 2 * idx + 1
    right = left + 1
    
    return np.concatenate([getChildren(left,N),getChildren(right,N)])

def treeNegativeLogLikelihood(x,leafN,normalize=True):
    def LL(leafMeans,bagSigma):
        NBags = len(bagSigma)
        NInternal_Nodes = np.floor(NBags/2)
        ll = 0
        for idx in range(NBags):
            leafIndices = (getChildren(idx, NInternal_Nodes) - NInternal_Nodes).astype(int)
            ln = leafN[leafIndices]
            mu = np.dot(leafMeans[leafIndices],ln)/np.sum(ln)
            sigma = bagSigma[idx]
            ll = ll + logLikelihood(x[idx],mu,sigma,normalize)
        return -1 * ll
    return LL

### Right now I'm assuming N = $2^j$ for some j

### Generate Data

In [ ]:
N = 7
N_Internal = int(np.floor((N)/2))
NLeaves = int(N - N_Internal)
bagMuStar = np.random.normal(loc=0,scale=10,size=NLeaves)
bagN = np.random.poisson(lam=10,size=NLeaves)

X = []
for level in range(3):
    NBagsInLevel = 2**level
    start = 2**level - 1
    for bagNum in range(start,start+NBagsInLevel):
        childrenIndices = (getChildren(bagNum,N_Internal) - N_Internal).astype(int)
        childrenMus = bagMuStar[childrenIndices]
        childrenNs = bagN[childrenIndices]
        loc = np.dot(childrenMus, childrenNs) / np.sum(childrenNs)
        scale = 2**level
        X.append(np.random.normal(loc=loc,scale=scale,size=np.sum(childrenNs)))

### Initialize as local estimates

In [ ]:
mu = np.zeros(bagMuStar.shape)
sigma = np.ones(len(X))
for leafNum in range(NLeaves):
    idx = N_Internal + leafNum
    xi = X[idx]
    mu[leafNum],sigma[idx] = ss.norm.fit(xi)
    

## Run Algorithm

In [ ]:
maes = []

gradNLL_mu = grad(treeNegativeLogLikelihood(X,bagN),0)
gradNLL_sigma = grad(treeNegativeLogLikelihood(X,bagN),1)
NIter= 1000
lr = 0.01
for i in tqdm(range(NIter),total=NIter):
    if not i % 5000:
        lr = lr * .5
    deltaMu = gradNLL_mu(mu,sigma)
    deltaSigma = gradNLL_sigma(mu,sigma)
    mu = mu - lr * deltaMu
    sigma = sigma - lr * deltaSigma
    maes.append(np.mean(np.abs(mu - bagMuStar)))


In [ ]:
plt.plot(maes)

In [ ]:
maes = []

gradNLL_mu = grad(treeNegativeLogLikelihood(X,bagN,normalize=False),0)
gradNLL_sigma = grad(treeNegativeLogLikelihood(X,bagN,normalize=False),1)
NIter= 5000
lr = 0.01
for i in tqdm(range(NIter),total=NIter):
    if not i % 5000:
        lr = lr * .5
    deltaMu = gradNLL_mu(mu,sigma)
    deltaSigma = gradNLL_sigma(mu,sigma)
    mu = mu - lr * deltaMu
    sigma = sigma - lr * deltaSigma
    maes.append(np.mean(np.abs(mu - bagMuStar)))



In [ ]:
plt.plot(maes)

# Try on real data

In [ ]:
from multiinstance.data.realData import buildDataset
from multiinstance.utils import *
from multiinstance.agglomerative_clustering import AgglomerativeClustering

In [ ]:
absErrs = {"local":[],
           "global":[],
           "likelihood":[]}

In [ ]:
fileNames = glob("/data/dzeiberg/ClassPriorEstimation/rawDatasets/*.mat")
for fileName in tqdm(fileNames,total=len(fileNames)):
    dsi = buildDataset(fileName,4,
                       alphaDistr=lambda: np.random.uniform(.01,.95),
                      nPDistr=lambda: 1 + np.random.poisson(100),
                      nUDistr=lambda: 1 + np.random.poisson(5000))

    dsi = addTransformScores(dsi)
    dsi = addGlobalEsts(dsi)
    dsi.alphaHats,dsi.curves = getBagAlphaHats(dsi,numbootstraps=50)

    dsi.numLeaves = dsi.alphaHats.shape[0]
    dsi.numNodes = dsi.numLeaves + (dsi.numLeaves - 1)
    dsi.numInternal = dsi.numNodes - dsi.numLeaves

    dsi.mu = np.zeros(dsi.alphaHats.shape[0])
    dsi.sigma = np.ones(dsi.numNodes)
    dsi.leafN = np.ones_like(dsi.mu) * dsi.alphaHats.shape[1]
    dsi.treeAlphaHats = [[] for _ in range(dsi.numNodes)]

    for nodeNum in range(dsi.numInternal):
        children = getChildren(nodeNum, dsi.numInternal)
        leafNums = children - dsi.numInternal
        pos,unlabeled = list(zip(*[getTransformScores(dsi,n) for n in leafNums]))
        pos = np.concatenate(pos).reshape((-1,1))
        unlabeled = np.concatenate(unlabeled).reshape((-1,1))
        NEstimates = int(np.sum([dsi.leafN[l] for l in leafNums]))
        dsi.treeAlphaHats[nodeNum],_ = getEsts(pos, unlabeled, NEstimates)

    for leafNum in range(dsi.numLeaves):
        nodeNum = leafNum + dsi.numInternal
        dsi.treeAlphaHats[nodeNum] = dsi.alphaHats[leafNum]
        dsi.mu[leafNum],dsi.sigma[nodeNum] = ss.norm.fit(dsi.treeAlphaHats[nodeNum])
    
    maes = [np.mean(np.abs(dsi.mu - dsi.trueAlphas.flatten()))]
    lr = 0.001

    gradNLL_mu = grad(treeNegativeLogLikelihood(dsi.treeAlphaHats,dsi.leafN),0)
    gradNLL_sigma = grad(treeNegativeLogLikelihood(dsi.treeAlphaHats,dsi.leafN),1)
    NIter= 5000
    for i in tqdm(range(NIter),total=NIter):
        if not i % 1500:
            lr = lr * .5
        deltaMu = gradNLL_mu(dsi.mu,dsi.sigma)
        deltaSigma = gradNLL_sigma(dsi.mu,dsi.sigma)
        dsi.mu = dsi.mu - lr * deltaMu
        dsi.sigma = dsi.sigma - lr * deltaSigma
        maes.append(np.mean(np.abs(dsi.mu - dsi.trueAlphas.flatten())))



    absErrs["local"].append(maes[0])
    absErrs["likelihood"].append(maes[-1])
    absErrs["global"].append(np.mean(np.abs(dsi.globalAlphaHats.mean() - dsi.trueAlphas.flatten())))

    plt.plot(maes)
    plt.hlines(absErrs["global"][-1],0,len(maes),color="black")
    plt.title(fileName.split("/")[-1])
    plt.show()

In [ ]:
dsi.globalAlphaHats.mean()

In [ ]:
dsi.sigma

In [ ]:
dsi.curves.shape

In [ ]:
plt.plot(dsi.curves[2,0])

In [ ]:
dsi.alphaHats

In [ ]:
dsi.trueAlphas[0]

### Final Results

In [ ]:
for k,v in absErrs.items():
    print(k, "{:.3f}".format(np.mean(v)))